In [14]:


# {year} {series ID} {Race ID}
year = 2025
series_id = 1
race_id = "ABC456"

In [15]:
import warnings
import requests
import pandas as pd

# endpoint for race list
#https://cf.nascar.com/cacher/2023/race_list_basic.json

class Schedule:
    
    def __init__(self, year, series_id):
        self.year = year
        self.series_id = series_id
        self.races = []
        self.data = pd.DataFrame()
        self.fetch_races()

    def fetch_races(self):
        """Fetch the race list for the specified year and series ID."""
        url = f"https://cf.nascar.com/cacher/{self.year}/race_list_basic.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Filter the races by series ID
            race_list = data[f'series_{self.series_id}']
            self.races = [race for race in race_list if race['series_id'] == self.series_id]
            self.data = pd.DataFrame(self.races)
            
        else:
            warnings.warn(f"Failed to fetch race list: {response.status_code}")
    def get_completed_races(self):
        """Return a list of completed races."""
        return self.data[self.data['winner_driver_id'].notna()]['race_name'].tolist()


schedule = Schedule(year, series_id)
schedule.data.head()

,race_id,series_id,race_season,race_name,race_type_id,restrictor_plate,track_id,track_name,date_scheduled,race_date,...,satellite_radio_broadcaster,master_race_id,inspection_complete,playoff_round,is_qualifying_race,qualifying_race_no,qualifying_race_id,has_qualifying,winner_driver_id,pole_winner_laptime
0,5543,1,2025,Cook Out Clash at Bowman Gray,2,False,159,Bowman Gray Stadium,2025-02-02T20:00:00,2025-02-02T20:00:00,...,SIRIUSXM,3222,True,0,False,0,-1,False,4062.0,None
1,5544,1,2025,Duel 1 at DAYTONA,2,True,105,Daytona International Speedway,2025-02-13T19:00:00,2025-02-13T19:00:00,...,SIRIUSXM,3217,True,0,False,0,-1,False,4025.0,None
2,5545,1,2025,Duel 2 at DAYTONA,2,True,105,Daytona International Speedway,2025-02-13T20:45:00,2025-02-13T20:45:00,...,SIRIUSXM,3218,True,0,False,0,-1,False,4180.0,None
3,5546,1,2025,DAYTONA 500,1,True,105,Daytona International Speedway,2025-02-16T14:30:00,2025-02-16T14:30:00,...,SIRIUSXM,385,True,0,False,0,-1,False,4184.0,None
4,5547,1,2025,Ambetter Health 400,1,True,111,Atlanta Motor Speedway,2025-02-23T15:00:00,2025-02-23T15:00:00,...,SIRIUSXM,3335,True,0,False,0,-1,False,4153.0,None


In [ ]:
#https://cf.nascar.com/cacher/2023/2/5314/lap-times.json
#

class Race:
    def __init__(self, year, series_id,race_id=None,live=False):
        # Initialize race metadata
        self.year = year
        self.live = live
        self.series_id = series_id
        self.race_id = race_id

        # Race Data
        self.name = None
        self.laps = None
        self.pit_stops = None
        self.start_time = None
        self.end_time = None
        self.winner = None
        self.data = {}
        self.fetch_laps()
        
    def fetch_laps(self):
        """Fetch lap times for the specified race ID."""
        if self.live:
            url = f"https://cf.nascar.com/cacher/live/series_{self.series_id}/{self.race_id}/lap-times.json"
        else:
            url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/lap-times.json"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            self.data = data
            lap_times = []
            
            for i in data['laps']:
                driver = i['FullName']
                number = i['Number']
                manufacturer = i['Manufacturer']
                for j in i['Laps']:
                    lap_times.append({
                        'Driver': driver,
                        'Number': number,
                        'Manufacturer': manufacturer,
                        'Lap': j['Lap'],
                        'lap_time': j['LapTime'],
                        'lap_speed': j['LapSpeed'],
                        'position': j['RunningPos'],
                    })
            self.laps = pd.DataFrame(lap_times)
            self.laps['Lap'] = self.laps['Lap'].astype(int)
            self.laps['lap_time'] = pd.to_timedelta(self.laps['lap_time'])
            self.laps['lap_speed'] = self.laps['lap_speed'].astype(float)
        else:
            print(f"Failed to retrieve lap data: {response.status_code}")
            print(f"url: {url}")

    def get_pit_stops(self):
        """Get pit stop information for the race."""
        if self.live:
            url = f"https://cf.nascar.com/cacher/live/series_{self.series_id}/{self.race_id}/live-pit-data.json"
        else:
            url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/live-pit-data.json"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            stops = []
            for i in data:
                stops.append({
                    'Driver': i.get('driver_name'),
                    'Lap': i.get('lap_count'),
                    'Manufacturer': i.get('Manufacturer'),
                    'StopTime': pd.to_timedelta(i.get('StopTime')) if i.get('StopTime') is not None else None,

                    # Added properties
                    'pit_in_flag_status': i.get('pit_in_flag_status'),
                    'pit_out_flag_status': i.get('pit_out_flag_status'),
                    'pit_in_race_time': i.get('pit_in_race_time'),
                    'pit_out_race_time': i.get('pit_out_race_time'),
                    'total_duration': i.get('total_duration'),
                    'box_stop_race_time': i.get('box_stop_race_time'),
                    'box_leave_race_time': i.get('box_leave_race_time'),
                    'pit_stop_duration': i.get('pit_stop_duration'),
                    'in_travel_duration': i.get('in_travel_duration'),
                    'out_travel_duration': i.get('out_travel_duration'),
                    'pit_stop_type': i.get('pit_stop_type'),
                    'left_front_tire_changed': i.get('left_front_tire_changed'),
                    'left_rear_tire_changed': i.get('left_rear_tire_changed'),
                    'right_front_tire_changed': i.get('right_front_tire_changed'),
                    'right_rear_tire_changed': i.get('right_rear_tire_changed'),
                    'previous_lap_time': i.get('previous_lap_time'),
                    'next_lap_time': i.get('next_lap_time'),
                    'pit_in_rank': i.get('pit_in_rank'),
                    'pit_out_rank': i.get('pit_out_rank'),
                    'positions_gained_lost': i.get('positions_gained_lost'),
                })
            # store both list and a DataFrame for convenience
            self.pit_stops = pd.DataFrame(stops) if stops else pd.DataFrame()
        else:
            print(f"Failed to retrieve pit stop data: {response.status_code}")
            print(f"url: {url}")

    def fetch_events(self):
        """ Fetch lap events and flags"""
        if self.live:
            url = f"https://cf.nascar.com/cacher/live/series_{self.series_id}/{self.race_id}/lap-notes.json"
        else:
            url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/lap-notes.json"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print(data)
        else:
            print(f"Failed to retrieve lap events: {response.status_code}")
            print(f"url: {url}")

race = Race(year, series_id, 5546)
race2 = Race(2023,2,5314)


In [49]:
race.get_pit_stops()
race.pit_stops

,Driver,Lap,Manufacturer,StopTime,pit_in_flag_status,pit_out_flag_status,pit_in_race_time,pit_out_race_time,total_duration,box_stop_race_time,...,pit_stop_type,left_front_tire_changed,left_rear_tire_changed,right_front_tire_changed,right_rear_tire_changed,previous_lap_time,next_lap_time,pit_in_rank,pit_out_rank,positions_gained_lost
0,William Byron,None,None,None,2,2,666.638,882.668,11614.947,676.235,...,OTHER,False,False,False,False,0,0,1,1,0
1,Austin Cindric,None,None,None,2,2,668.834,885.746,11615.829,696.015,...,TWO_WHEEL_CHANGE_RIGHT,False,False,True,True,0,0,2,2,0
2,Ty Dillon,None,None,None,2,2,669.462,887.356,11616.811,688.815,...,OTHER,False,False,False,False,0,0,3,3,0
3,Chase Briscoe,None,None,None,2,2,670.433,889.714,11618.198,703.315,...,OTHER,False,False,False,False,0,0,4,4,0
4,Joey Logano,None,None,None,2,2,672.424,903.406,11629.899,700.895,...,FOUR_WHEEL_CHANGE,True,True,True,True,0,0,6,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,Todd Gilliland,None,None,None,2,2,13654.684,13767.783,113.099,13666.650,...,TWO_WHEEL_CHANGE_RIGHT,False,False,True,True,0,0,31,31,0
332,Chase Elliott,None,None,None,2,2,13650.951,13688.262,37.311,13668.130,...,FOUR_WHEEL_CHANGE,True,True,True,True,0,0,17,20,-3
333,* Justin Allgaier(i),None,None,None,2,2,13651.437,13735.297,83.860,13668.850,...,FOUR_WHEEL_CHANGE,True,True,True,True,0,0,20,24,-4
334,Carson Hocevar,None,None,None,2,0,13703.291,-1.000,300.000,13710.990,...,FOUR_WHEEL_CHANGE,True,True,True,True,0,0,28,28,0


In [44]:
race.laps

,Driver,Number,Manufacturer,Lap,lap_time,lap_speed,position
0,William Byron,24,Chv,0,NaT,NaN,5
1,William Byron,24,Chv,1,0 days 00:00:00.000000052,172.629,5
2,William Byron,24,Chv,2,0 days 00:00:00.000000047,188.045,5
3,William Byron,24,Chv,3,0 days 00:00:00.000000047,187.633,4
4,William Byron,24,Chv,4,0 days 00:00:00.000000047,187.668,2
...,...,...,...,...,...,...,...
7474,AJ Allmendinger,16,Chv,38,0 days 00:00:00.000000047,189.498,12
7475,AJ Allmendinger,16,Chv,39,0 days 00:00:00.000000047,189.139,15
7476,AJ Allmendinger,16,Chv,40,0 days 00:00:00.000000047,189.406,14
7477,AJ Allmendinger,16,Chv,41,0 days 00:00:00.000000047,189.450,14


In [7]:
laps = race.test['laps']
lap_times = []

for i in laps:
    driver = i['FullName']
    number = i['Number']
    manufacturer = i['Manufacturer']
    for j in i['Laps']:
        lap_times = lap_times
        lap_times.append({
            'Driver': driver,
            'Number': number,
            'Manufacturer': manufacturer,
            'Lap': j['Lap'],
            'lap_time': j['LapTime'],
            'lap_speed': j['LapSpeed'],
            'position': j['RunningPos'],
        })
        
lap_times = pd.DataFrame(lap_times)
lap_times

TypeError: 'NoneType' object is not subscriptable

In [18]:
race.laps.Driver.unique()

AttributeError: 'NoneType' object has no attribute 'Driver'